In [ ]:
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import time
import matplotlib.pyplot as plt